Bài thực hành giới thiệu về dự đoán dữ liệu dạng chuỗi thời gian bằng RNN, gồm 2 phần: dự đoán trên chuỗi thời gian đơn biến (có 1 đặc trưng) và dự đoán trên chuỗi thời gian đa biến (có nhiều đặc trưng).

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## Tập dữ liệu thời tiết
Tập dữ liệu về thời tiết [này](https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip) bao gồm 14 đặc trưng khác nhau như nhiệt độ không khí, áp suất không khí, độ ẩm,... ở Jena, Đức, được ghi nhận cứ mỗi 10 phút, bắt đầu từ năm 2003 đến 2016.

In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
csv_path

In [0]:
df = pd.read_csv(csv_path)
df.tail(5)

In [0]:
# test = pd.read_excel("/content/VN30.xls")
# test.head()

Ví dụ nếu ta muốn dự đoán nhiệt độ của 6 giờ tiếp theo dựa trên dữ liệu của 3 giờ gần nhất, ta sẽ tạo ra các cửa sổ, mỗi cửa sổ gồm 18 (3 x 6) mẫu dữ liệu.

Hàm bên dưới sẽ giúp ta thực hiện điều đó với `history_size` là kích thước của cửa sổ (18), target_size là số dự đoán tương lai mà chúng ta muốn (6).

In [0]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

Ta chọn 300k dòng đầu tiên làm tập huấn luyện (khoảng 2083 ngày). Phần còn lại là tập kiểm thử.

In [0]:
TRAIN_SPLIT = 300000

Thiết lập random seed để đảm bảo có thể tái tạo lại kết quả.

In [0]:
tf.random.set_seed(45)

## Phần 1. Dự đoán bằng chuỗi thời gian đơn biến

Ta sẽ huấn luyện mô hình chỉ bằng một đặc trưng duy nhất là `nhiệt độ` để từ đó đưa ra dự đoán về nhiệt độ trong tương lai.

In [0]:
uni_data = df['T (degC)']
uni_data.index = df['Date Time']
uni_data.head()

Vẽ đồ thị để quan sát nhiệt độ bằng `.plot`

In [0]:
uni_data.plot(subplots=True)

In [0]:
uni_data = uni_data.values
uni_data

Chuẩn hóa dữ liệu


Lưu ý: Bước tính trung bình, độ lệch chuẩn (std) chỉ nên được tính toán bằng dữ liệu huấn luyện.

In [0]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

In [0]:
uni_data = (uni_data - uni_train_mean) / uni_train_std

Ở đây, ta sẽ dự đoán nhiệt độ của bước thời gian tiếp theo (time step), bằng 20 mẫu thời gian trước đó.

In [0]:
univariate_past_history = 20
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [0]:
print ('Ví dụ về ô cửa sổ dữ liệu')
print (x_train_uni[0])
print ('\n Mục tiêu mà ta sẽ dự đoán')
print (y_train_uni[0])

In [0]:
print ('Ví dụ về ô cửa sổ dữ liệu')
print (x_train_uni[1])
print ('\n Mục tiêu mà ta sẽ dự đoán')
print (y_train_uni[1])

Vẽ biểu đồ biểu diễn tập huấn luyện.

In [0]:
def create_time_steps(length):
  time_steps = []
  for i in range(-length, 0, 1):
    time_steps.append(i)
  return time_steps

In [0]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data): #return (index, value)
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [0]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example')

### Baseline
Trước khi bắt đầu thiết kế mô hình, ta hãy thử dự đoán nhiệt độ hiện tại bằng cách tính trung bình 20 mức nhiệt độ trước đó.

In [0]:
def baseline(history):
  return np.mean(history)

In [0]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example')

### Long short term memory


In [0]:
#Ta dùng tf.data để trộn ngẫu nhiên dữ liệu, phân lô và lưu tạm (cache) tập dữ liệu
#https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset
BATCH_SIZE = 64
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE)

The following visualisation should help you understand how the data is represented after batching.

![Time Series](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/images/time_series.png?raw=1)

Mô hình RNN sẽ yêu cầu ta khai báo kích cỡ đầu vào (input shape) theo dạng (số kích thước cửa số, số dự đoán).

Ở đây chính là (20, 1)

In [0]:
x_train_uni.shape

In [0]:
x_train_uni.shape[-2:]

In [0]:
simple_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(16, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_model.compile(optimizer='adam', loss='mae')

In [0]:
EVALUATION_INTERVAL = 400
EPOCHS = 10

simple_model.fit(train_univariate, epochs=EPOCHS,
                      validation_data=val_univariate)

#### Dự đoán và vẽ đồ thị

In [0]:
for x, y in val_univariate.take(3):
  plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_model.predict(x)[0]], 0, 'Simple model')
  plot.show()

## Phần 2: Dự đoán bằng chuỗi thời gian đa biến

Ở phần 2 này, ta sẽ dự đoán nhiệt độ bằng 3 đặc trưng: áp suất, nhiệt độ và mật độ khí quyển.

In [0]:
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

In [0]:
features = df[features_considered]
features.index = df['Date Time']
features.head()

Vẽ biểu đồ để xem các đặc trưng thay đổi như thế nào theo thời gian.

In [0]:
features.plot(subplots=True)

In [0]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)

In [0]:
dataset = (dataset-data_mean)/data_std

### Mô hình dự đoán 1 điểm nhiệt độ mới
Ở phần này, mô hình sẽ học và dự đoán một điểm nhiệt độ mới dựa trên những điểm dữ liệu (nhiệt độ, áp suất, mật độ khí quyển) đã biết.

In [0]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

Ta sẽ dự đoán điểm nhiệt độ mới từ dữ liệu của 5 ngày trước đó (5 x 24 x 6) là 720 điểm dữ liệu. 

Do trong vòng 1 giờ có thể sẽ không có quá nhiều biến động nên ta sẽ chỉ lấy những điểm dữ liệu cách nhau 1 giờ. 720 / 6 = 120 điểm.

Ta muốn dự đoán nhiệt độ của 12 giờ tới. Do đó mục tiêu sẽ là điểm cách 12 x 6 = 72 bước so với hiện tại.

In [0]:
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

Xem thử điểm dữ liệu

In [0]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [0]:
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE)

In [0]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')

In [0]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            validation_data=val_data_single)

In [0]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [0]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

#### Dự đoán


In [0]:
for x, y in val_data_single.take(3):
  plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Single Step Prediction')
  plot.show()

### Mô hình dự đoán nhiều điểm dữ liệu mới

Ở đây ta muốn dựa trên những dữ liệu đã biết, ta sẽ dự đoán ra một chuỗi các điểm nhiệt độ mới.

In [0]:
future_target = 72
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

Xem thử dữ liệu

In [0]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train_multi[0].shape))

In [0]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

Định nghĩa hàm vẽ đồ thị.

In [0]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

Kiểm tra đồ thị

In [0]:
for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

Ta sẽ thử xem liệu 1 tầng LSTM có đủ đẻ giải quyết bài toán này hay không.

In [0]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(72))

multi_step_model.compile(optimizer='adam', loss='mae')

In [0]:
EVALUATION_INTERVAL = 100
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [0]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

#### Dự đoán

In [0]:
for x, y in val_data_multi.take(3):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])